# Process - SocialDF

In [33]:
import polars as pl
import re
from src.utils.download_insta_vids import download_insta_vids


real = pl.read_csv("../socialdf_real.csv").rename({"officialblazequotes\n":"url"}).with_columns(pl.lit(0).alias("target"))
fake = pl.read_csv("../socialdf_deepfake.csv").with_columns(pl.lit(1).alias("target")).rename({"DEEPFAKE DATA LINKS":"url"})
real.head()

url,target
str,i32
"""https://www.instagram.com/reel…",0
"""https://www.instagram.com/p/DB…",0
"""https://www.instagram.com/p/Cx…",0
"""https://www.instagram.com/p/Cs…",0
"""https://www.instagram.com/p/DE…",0


In [42]:
df = pl.concat([real, fake])
ids = []
urls = df["url"].to_list()
for u in urls:
    try:
        match = re.search(r"instagram\.com/p/([a-zA-Z0-9_-]+)", u)
        ids.append(match.group(1))
    except:
        ids.append(None)
        
df = df.with_columns(pl.Series("id", ids)).filter(pl.col("id").is_not_null())


In [ ]:
urls = df["url"].to_list()
for u in urls:
    download_insta_vids(u, output_folder="vids")

In [43]:
import os
ids_downloaded = [f for f in os.listdir("vids")]
df = df.filter(pl.col("id").is_in(ids_downloaded))
df.write_csv("socialdf_processed.csv")
df.group_by("target").count()

/tmp/ipykernel_48320/874786809.py:5: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  df.group_by("target").count()


target,count
i32,u32
0,461
1,124


In [41]:
ids_downloaded

['C',
 'DEYK2ErNHmV',
 'C9QBCdrtpjj',
 'DDSYuOGN0Yh',
 'DDcb',
 'CuZOthmLcp0',
 'C-OECRENQ1k',
 'DCHoYekt-v0',
 'C27ynRqPbnJ',
 'C90Xlt2MNqV',
 'DCuVUU7t5rI',
 'C6PMJfhLu1I',
 'C9mNSBzor7n',
 'C8H-Qa3tk6r',
 'C7ow3nBtPOb',
 'DDzhvZJNwn1',
 'DCChN9utiHA',
 'DEP3BuztxjJ',
 'C7HDRJSqMaW',
 'C',
 'C7I-Mq6Kx4Y',
 'C7Ywj6soi7q',
 'C8w',
 'DEAx8sUNxB5',
 'DEIkPxKtW3I',
 'Cw3JQRZNBCO',
 'CvrlG',
 'C',
 'DDXgaRCKU8c',
 'DDulha-t85Y',
 'C8z4Xm2tddf',
 'DAwErZytl3y',
 'C-LidNhoC-a',
 'DCzerhaNUea',
 'C9cynmTtIcx',
 'DC4',
 'DDH5iPeqSjW',
 'DCUabVetnjr',
 'DDNLYk9trox',
 'C8Pm2mmNwD',
 'CyrbPXzNNxN',
 'Ct-CfVFtv24',
 'C--6pRxtl7F',
 'C9p4zgMtrFF',
 'C6MLPG1Kh33',
 'DAb1Wo6Os4c',
 'DBEy3fBtNnm',
 'Cs3hVw3vmWf',
 'DC4NqerNZ9L',
 'C7MK4Q7ojNz',
 'DB9Pa17t0ll',
 'C-GD9jDNx4j',
 'CxTcjWEtB',
 'C8gn1b9ohgC',
 'DA9M8d5Nyun',
 'C',
 'C8AJgoOt-B3',
 'C',
 'DDp2BMht6JX',
 'C-YYlHTI39s',
 'DDVXrsGNxDR',
 'DBzFdE-N7DE',
 'C9LG',
 'DC9jT6ztHoU',
 'C-vI47wtG1B',
 'DBCBHadNAM2',
 'C8cT2jAotA4',
 'C',
 'CvaGwJsOT